In [1]:
import os
import sys
sys.path.append('../../src')
import pandas as pd
import numpy as np
from tqdm import tqdm
import random
from datetime import datetime
import time
from hydra import initialize, compose
import pathlib
# import config

import data_process.neg_sample as ng_sample
from data_process.utils import mix_merge
from data_process.data_split import data_split_user
from evaluate_entity import CustomHR, CustomNDCG, CustomRoc, CustomRoctop, CustomRecall_top, CustomPrecision_top
from model_entity import EntityCat
from data_utils import CatData
from utils.constants import DEFAULT_USER_COL,DEFAULT_ITEM_COL,DEFAULT_RATING_COL, DEFAULT_TIMESTAMP_COL


from sklearn import metrics, preprocessing
from sklearn.metrics import roc_auc_score, average_precision_score, recall_score, precision_score

import torch
import torch.nn as nn
import torch.utils.data as data
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.tensorboard import SummaryWriter

# import argparse
torch.manual_seed(0)

/Users/hao/.pyenv/versions/3.6.15/envs/torch-cpu5/lib/python3.6/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from ignite.engine import Engine, Events, create_supervised_trainer, create_supervised_evaluator, RemovableEventHandle
from ignite.metrics import Accuracy, Loss, Metric
from ignite.handlers import ModelCheckpoint, EarlyStopping
from ignite.exceptions import NotComputableError
from ignite.metrics.metric import sync_all_reduce, reinit__is_reduced
from ignite.contrib.handlers.tqdm_logger import ProgressBar
# from ignite.contrib.handlers import TensorboardLogger 
from ignite.contrib.handlers.wandb_logger import *

In [3]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device

device(type='cpu')

In [4]:
with initialize(version_base=None, config_path="../conf"):
    cfg = compose(config_name="config", overrides=[])

In [5]:
if device.type =='cpu':
    BATCH_SIZE = cfg.params.batch_size_cpu
    EPOCHS  = cfg.params.epochs_cpu
else:
    BATCH_SIZE = cfg.params.batch_size_gpu
    EPOCHS  = cfg.params.epochs_gpu

In [6]:
if device.type == 'cpu':
    use_amp=False
    df_train_pos  = ng_sample.read_feather(pathlib.Path(cfg.path.root, cfg.file.train_pos))
    df_train_neg = pd.read_feather(pathlib.Path(cfg.path.root, cfg.file.train_neg))
    df_test_ori = pd.read_feather(pathlib.Path(cfg.path.root, cfg.file.test)).iloc[:202,]
    df_all_features = pd.read_csv(pathlib.Path(cfg.path.root, cfg.file.all_features))
    df_train_pos = df_train_pos.sort_values(by=[DEFAULT_USER_COL]).iloc[:100,].reset_index(drop=True)
    df_train_neg = df_train_neg.sort_values(by=[DEFAULT_USER_COL]).iloc[:100*cfg.params.neg_train,].reset_index(drop=True)
else:
    use_amp=True
    df_train_pos  = ng_sample.read_feather(pathlib.Path(cfg.path.root, cfg.file.train_pos))
    df_train_neg = pd.read_feather(pathlib.Path(cfg.path.root, cfg.file.train_neg))
    df_test_ori = pd.read_feather(pathlib.Path(cfg.path.root, cfg.file.test))
    df_all_features = pd.read_csv(pathlib.Path(cfg.path.root, cfg.file.all_features))
    df_train_pos = df_train_pos.sort_values(by=[DEFAULT_USER_COL]).reset_index(drop=True)
    df_train_neg = df_train_neg.sort_values(by=[DEFAULT_USER_COL]).reset_index(drop=True)

In [7]:
df_train_pos[DEFAULT_RATING_COL] = 1

In [8]:
def concat_index(df1, df2):
    df2.index = df2.index//cfg.params.neg_train
    return pd.concat([df1, df2], axis=0).sort_index(kind='mregesort').reset_index(drop=True)

In [9]:
df_train_all = concat_index(df_train_pos, df_train_neg)

In [10]:
df_train_all['flag'] = 1
df_test_ori['flag'] = 0
df_all = pd.concat([df_train_all, df_test_ori], axis=0).reset_index(drop=True)

user features: 
       'WindowID_user', 'Split', 'City',
       'State', 'Country', 'Zip_user', 'DegreeType', 'Major', 'GraduationDate',
       'WorkHistoryCount', 'TotalYearsExperience', 'CurrentlyEmployed',
       'ManagedOthers', 'ManagedHowMany',
       
job features: 
       'WindowID_job', 'City_job',
       'State_job', 'Country_job', 'Zip_job', 'StartDate', 'EndDate',

### Choose the features and process data for the training

In [11]:
df_all.columns

Index(['userid', 'itemid', 'rating', 'flag'], dtype='object')

In [12]:
user_features = ['City']
user_features_extend = [DEFAULT_USER_COL] + user_features

item_features = ['City_job']
item_features_extend =[DEFAULT_ITEM_COL] + item_features

base_features = [DEFAULT_USER_COL, DEFAULT_ITEM_COL, DEFAULT_RATING_COL]

In [13]:
df_mix_merge = mix_merge(df_all , df_all_features, user_features_extend, item_features_extend)

In [14]:
def _cat_encode(df_data, list_f, encoder):
    for f in list_f:
        df_data[f] = encoder.fit_transform(df_data[f].astype('category').cat.codes.values)
    return df_data

In [15]:
def _embedding_dimension(df_all_encode, features_to_train, max_dim=50):

    embedding_size = []
    features_to_em = [i for i in features_to_train if i !=DEFAULT_RATING_COL]
    for c in features_to_em:
        num_unique_values = int(df_all_encode[c].nunique())
        embed_dim = int(min(np.ceil(num_unique_values/2), max_dim))
        embedding_size.append([num_unique_values, embed_dim])  
    return embedding_size

In [16]:
def encode_data(df_mix_merge, features_to_code, features_to_train, max_dim=50):
    encoder = preprocessing.LabelEncoder()
    df_all_encode = _cat_encode(df_mix_merge, features_to_code, encoder)
    df_train = df_all_encode[df_all.flag==1]
    df_test = df_all_encode[df_all.flag==0]
    df_train = df_train[features_to_train]
    df_test = df_test[features_to_train]
    embedding_size = _embedding_dimension(df_all_encode, features_to_train, max_dim)
    return df_train, df_test, embedding_size

In [17]:
num_feature=[]
features_to_code = df_mix_merge.columns
features_to_train = [DEFAULT_USER_COL, DEFAULT_ITEM_COL]+ user_features + item_features +[DEFAULT_RATING_COL]


In [18]:
df_train,  df_test, embedding_size = encode_data(df_mix_merge, features_to_code, features_to_train, max_dim=cfg.params.emb_dim)

print(f'The size of embedding layers:{embedding_size}')

The size of embedding layers:[[36, 18], [413, 50], [35, 18], [230, 50]]


## Run data check before training 

Check the ratio of positive and negative samples

In [19]:
assert len(df_train[df_train.rating==0])/len(df_train[df_train.rating==1]) == cfg.params.neg_train, 'wrong neg/pos ratio in training set'
assert len(df_test[df_test.rating==0])/len(df_test[df_test.rating==1]) == cfg.params.neg_test, 'wrong neg/pos ratio in test set '
#Check if all the users in test can be found in training set
assert sum(np.isin(df_test.userid.unique(), df_train.userid.unique(), assume_unique=True)) == len(df_test.userid.unique()), 'cold start'
#The the uniqueness of items between training and test. For a user, on common items between training and test dataset. 
assert df_all.shape[0] ==df_train.shape[0]+df_test.shape[0], 'wrong data concat'
assert sum(df_all.groupby(['userid']).apply(lambda x: len(x['itemid'].unique()))) == df_all.shape[0], 'train and test have overlap item'

## Creat the numpy array for training 

In [20]:
df_train_split, df_val_split = data_split_user(df_train, val_size=0.2)

np_train = df_train_split.values
np_val = df_val_split.values
np_test = df_test.values

In [21]:
def seed_worker(worker_id):
    worker_seed = torch.initial_seed() % 2**32
    numpy.random.seed(worker_seed)
    random.seed(worker_seed)

g = torch.Generator()
g.manual_seed(0)

In [22]:
train_dataset = CatData(np_train)
val_dataset = CatData(np_val)
test_dataset = CatData(np_test) 
train_loader = data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=0,  worker_init_fn=seed_worker,generator=g)
val_loader = data.DataLoader(val_dataset, batch_size=len(val_dataset), shuffle=False, num_workers=0,  worker_init_fn=seed_worker,generator=g)
test_loader = data.DataLoader(test_dataset, batch_size=cfg.params.neg_test+1, shuffle=False, num_workers=0,worker_init_fn=seed_worker,generator=g )

In [23]:
model = EntityCat(embedding_size = embedding_size, num_numerical_cols = len(num_feature),
               output_size = 1)
model.to(device)

EntityCat(
  (all_embeddings): ModuleList(
    (0): Embedding(36, 18)
    (1): Embedding(413, 50)
    (2): Embedding(35, 18)
    (3): Embedding(230, 50)
  )
  (mlp_layers): Sequential(
    (0): Linear(in_features=136, out_features=100, bias=True)
    (1): ReLU(inplace=True)
    (2): Dropout(p=0.4, inplace=False)
  )
  (predict_layer): Linear(in_features=100, out_features=1, bias=True)
)

In [24]:
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=cfg.params.lr)

## Model the training 

In [25]:
def output_trans_loss(output):
    return output['y_pred'], output['label']

val_metrics_train = {
    'auc': CustomRoc(),
    "loss": Loss(criterion, output_transform=output_trans_loss)
}

val_metrics_test = {
    'hr': CustomHR(),
    'ndcg': CustomNDCG(),
    'auc': CustomRoc(),
    'roc_top': CustomRoctop(),
    'recall_top': CustomRecall_top(threshold=0.5),
    'precision_top': CustomPrecision_top(threshold=0.5),
    "loss": Loss(criterion, output_transform=output_trans_loss)
}

In [65]:
scaler = torch.cuda.amp.GradScaler(enabled=use_amp)
def train_step(engine, batch):
    model.train()
    optimizer.zero_grad()
    x, y = batch[0].to(device), batch[1].to(device)
    with torch.cuda.amp.autocast(enabled=use_amp):
        y_pred = model(x).reshape(1,-1).flatten()
        loss = criterion(y_pred, y.float())
    scaler.scale(loss).backward()
    scaler.step(optimizer)
    scaler.update()
    return loss.item()

def validation_step(engine, batch):
    model.eval()
    with torch.no_grad():
        x, label = batch[0].to(device), batch[1].to(device)
        y_pred = model(x).reshape(1,-1).flatten()
        label=label.float()
        return {'label':label, 'y_pred':y_pred}

def test_step(engine, batch):
    model.eval()
    with torch.no_grad():
        x, label = batch[0].to(device), batch[1].to(device)
        y_pred = model(x).reshape(1,-1).flatten()
        label=label.float()
        y_pred_top, indices = torch.topk(y_pred, engine.state.topk)
        y_pred_top = y_pred_top.detach().cpu().numpy()
        reco_item = torch.take(x[:,1], indices).cpu().numpy().tolist()
        pos_item = x[0,1].cpu().numpy().tolist()  # ground truth, item id
        label_top = label[indices].cpu().numpy()
        return {'pos_item':pos_item, 'reco_item':reco_item, 'y_pred_top':y_pred_top, 'label_top':label_top, 'label':label, 'y_pred':y_pred}

trainer = Engine(train_step)

train_evaluator = Engine(validation_step)
# train_evaluator.state_dict_user_keys.append('topk')

val_evaluator = Engine(validation_step)
# val_evaluator.state_dict_user_keys.append('topk')

test_evaluator = Engine(test_step)
test_evaluator.state_dict_user_keys.append('topk')

# @val_evaluator.on(Events.STARTED)
# def init_user_value():
#     val_evaluator.state.topk=3
    
# @train_evaluator.on(Events.STARTED)
# def init_user_value():
#     train_evaluator.state.topk=3

@train_evaluator.on(Events.STARTED)
def init_user_value():
    test_evaluator.state.topk=cfg.params.topk
    
    
# Attach metrics to the evaluators
for name, metric in val_metrics_train.items():
    metric.attach(train_evaluator, name)

for name, metric in val_metrics_train.items():
    metric.attach(val_evaluator, name)

    
for name, metric in val_metrics_test.items():
    metric.attach(test_evaluator, name)
    
# Eearly_stop 
def score_function(engine):
    val_loss = engine.state.metrics['auc']
    return val_loss

Eearly_stop_handler = EarlyStopping(patience=cfg.params.patience, score_function=score_function, trainer=trainer)
val_evaluator.add_event_handler(Events.COMPLETED, Eearly_stop_handler)

In [66]:
@trainer.on(Events.EPOCH_COMPLETED)
def log_training_results(trainer):
    train_evaluator.run(train_loader)
    metrics = train_evaluator.state.metrics
    auc = metrics['auc']
    loss = metrics['loss']
    print(f'Training Results- Epoch[{trainer.state.epoch}]  Avg loss: {loss:.2f} \
          Avg auc:{auc:.2f}')


@trainer.on(Events.EPOCH_COMPLETED)
def log_validation_results(trainer):
    val_evaluator.run(val_loader)
    metrics = val_evaluator.state.metrics
    auc = metrics['auc']
    loss = metrics['loss']
    print(f'Validation Results- Epoch[{trainer.state.epoch}]  Avg loss: {loss:.2f} \
          Avg auc:{auc:.2f}')

    
@trainer.on(Events.COMPLETED)
def log_test_results(trainer):
    test_evaluator.run(test_loader)
    metrics = test_evaluator.state.metrics
    hr = metrics['hr']
    ndcg = metrics['ndcg']
    auc = metrics['auc']
    roc_top = metrics['roc_top']
    recall = metrics['recall_top']
    precision = metrics['precision_top']
    loss = metrics['loss']
    print(f"Test Results - Epoch[{trainer.state.epoch}]  Avg loss: {loss:.2f} \
     Avg ndcg: {ndcg:.2f}  Avg auc: {auc:.2f}  Avg auc_top: {roc_top:.2f} \
      Avg recall: {recall:.2f}  Avg precision: {precision:.2f}")

pbar = ProgressBar(persist=False)
pbar.attach(trainer)

In [73]:
trainer.run(train_loader, max_epochs=2)

In [30]:
config_dict = dict(cfg.params)
config_dict['Features']='-'.join(user_features+item_features)
# wandb_logger = WandBLogger(
#     project="pytorch-jrs",
#     name="-".join(user_features)+'-'+'-'.join(item_features),
#     config=config_dict,
#     tags=["entity", "jrs"]
# )

to_save = {'model': model}
checkpoint_handler = ModelCheckpoint(
    wandb_logger.run.dir,
    n_saved=1, filename_prefix='best',
    score_name="auc",
    global_step_transform=global_step_from_engine(trainer)
)

val_evaluator.add_event_handler(Events.COMPLETED, checkpoint_handler, to_save)

    
wandb_logger.attach_output_handler(
    trainer,
    event_name=Events.ITERATION_COMPLETED,
    tag="training",
    output_transform=lambda loss: {"loss": loss}
)

wandb_logger.attach_output_handler(
    train_evaluator,
    event_name=Events.EPOCH_COMPLETED,
    tag="training",
    metric_names=['loss','auc'],
    global_step_transform=lambda *_: trainer.state.iteration,
)

wandb_logger.attach_output_handler(
    val_evaluator,
    event_name=Events.EPOCH_COMPLETED,
    tag="validation",
    metric_names=['loss',"auc"],
    global_step_transform=lambda *_: trainer.state.iteration,
)


wandb_logger.attach_output_handler(
    test_evaluator,
    event_name=Events.COMPLETED,
    tag="test",
    metric_names=['loss',"auc", 'hr', 'ndcg', 'roc_top', 'recall_top', 'precision_top'],
    global_step_transform=lambda *_: trainer.state.iteration,
)


wandb_logger.attach_opt_params_handler(
    trainer,
    event_name=Events.ITERATION_STARTED,
    optimizer=optimizer,
    param_name='lr'  # optional
)

# wandb_logger.watch(model) 

# trainer.run(train_loader, max_epochs=EPOCHS)
wandb_logger.close()

wandb: Currently logged in as: tiyuok2023. Use `wandb login --relogin` to force relogin


## Quantity evaluation 
display the recommended items for the user. Sample the recommended test users 

In [75]:
unique_users_test = df_test[DEFAULT_USER_COL].unique()
np.random.seed(123)
sub_users_test = np.random.choice(unique_users_test, 2, replace=False)

In [76]:
df_test_sub = df_test[df_test[DEFAULT_USER_COL].isin(sub_users_test)]

In [77]:
sample_index =df_test[DEFAULT_USER_COL].isin(sub_users_test).index

In [78]:
df_test_sub_ori = df_all[df_all.index.isin(sample_index)]

In [79]:
np_test_sub = df_test_sub.values

In [80]:
test_dataset_sample = CatData(np_test_sub)
test_sample_loader = data.DataLoader(test_dataset_sample, batch_size=cfg.params.neg_test+1, shuffle=False, num_workers=0,worker_init_fn=seed_worker,generator=g)

In [81]:
class RecoList(Metric):
    '''
    The recommendation list 
    '''
    def __init__(self, output_transform=lambda x: [x['pos_item'], x['reco_item'], x['userid'], x['y_pred_top'], x['indices']], device="cpu"):
#         self._reco_dict = []
        self.item= np.array([])
        self.userid= np.array([])
        self.prob= np.array([])
        self.indices = []
#         self.cnt = 0
        super(RecoList, self).__init__(output_transform=output_transform, device=device)

    @reinit__is_reduced
    def reset(self):
        self.item= np.array([])
        self.userid= np.array([])
        self.prob= np.array([])
#         self.cnt = 0
        super(RecoList, self).reset()

    @reinit__is_reduced
    def update(self, output):
        gt_item = output[0]
        reco_item = output[1]
        userid = output[2]
        prob = output[3]
        indices = output[4]
        self.item = np.append(self.item, reco_item)
        self.userid = np.append(self.userid, userid)
        self.prob = np.append(self.prob, prob)
        self.indices.append(indices)
        
    @sync_all_reduce("_reco_dict")
    def compute(self):
        return self.indices, self.prob

In [82]:
val_metrics_test_quanlity = {
    'recolist': RecoList()
}

In [83]:
def test_q_step(engine, batch):
    model.eval()
    with torch.no_grad():
        x, label = batch[0].to(device), batch[1].to(device)
        y_pred = model(x).reshape(1,-1).flatten()
#         print(y_pred)
        label=label.float()
        y_pred_top, indices = torch.topk(y_pred, engine.state.topk)
        y_pred_top = y_pred_top.cpu().numpy().tolist()
        y_pred_top = y_pred_top
        reco_item = torch.take(x[:,1], indices).cpu().numpy().tolist()
        pos_item = x[0,1].cpu().numpy().tolist()  # ground truth, item id 
        userid = x[:engine.state.topk,0].cpu().numpy().tolist()
        return {'reco_item': reco_item, 'pos_item':pos_item, 'userid':userid, 'y_pred_top':y_pred_top, 'indices':indices.cpu().numpy().tolist()}

In [84]:
test_q_evaluator=Engine(test_q_step)
for name, metric in val_metrics_test_quanlity.items():
    metric.attach(test_q_evaluator, name)

In [85]:
test_q_evaluator.state_dict_user_keys.append('topk')

@test_q_evaluator.on(Events.STARTED)
def init_user_value():
    test_q_evaluator.state.topk=cfg.params.topk

In [86]:
test_q_evaluator.run(test_sample_loader)

State:
	iteration: 2
	epoch: 1
	epoch_length: 2
	max_epochs: 1
	output: <class 'dict'>
	batch: <class 'list'>
	metrics: <class 'dict'>
	dataloader: torch.utils.data.dataloader.DataLoader
	seed: <class 'NoneType'>
	times: <class 'dict'>
	topk: 10

In [87]:
bb, prob =test_q_evaluator.state.metrics['recolist']

In [89]:
# prob

In [90]:
# df_test_sub_ori

In [91]:
len_train = len(df_train_all)

In [92]:
reco_indices = []
for i in range(len(bb)):
    reco_indices += [cfg.params.neg_test*(i)+j for j in bb[i]]

In [93]:
df_reco = df_test_sub_ori.iloc[reco_indices].copy()

In [94]:
df_reco['prob'] = prob

In [96]:
df_reco.head()

,userid,itemid,rating,flag,prob
578,7,732328,0,0,0.432161
534,7,382423,0,0,0.416606
533,7,725768,0,0,0.413199
500,7,703889,1,0,0.412339
582,7,663485,0,0,0.398364


In [97]:
job_info = pd.read_csv('../../data/jobs/jobid_title.csv')

In [98]:
job_info.rename(columns={'item_id':'itemid'}, inplace=True)

In [99]:
job_info

,Unnamed: 0,itemid,Title
0,0,1,Security Engineer/Technical Lead
1,1,4,SAP Business Analyst / WM
2,2,7,P/T HUMAN RESOURCES ASSISTANT
3,3,8,Route Delivery Drivers
4,4,9,Housekeeping
...,...,...,...
1091918,1091918,1116242,ELECTRICAL SPECIALIST
1091919,1091919,1116253,General and Assistant Managers
1091920,1091920,1116294,Assistant Retail Store Manager - Retail Sales ...
1091921,1091921,1116304,Career Experience Specialist - Career Services...


In [100]:
pd.merge(df_reco, job_info, how='left', on={'itemid'})

KeyError: {'itemid'}